In [ ]:
from dataprocessing import generate_tasks
from fitting import get_model, fit
from meta import Meta
import optuna as opt
import numpy as np
import wandb
import torch

In [ ]:
def optimize(trial, train_signals, val_signals, bkg_file, args):
    # Manually seed torch and numpy for reproducible results
    torch.manual_seed(args["seed"])
    torch.cuda.manual_seed_all(args["seed"])
    np.random.seed(args["seed"])
    
    # Defining trial parameters
    sup_shots = trial.suggest_int("k_sup", 100, 100)
    que_shots = trial.suggest_int("k_que", 200, 200)
    num_layers = trial.suggest_int("num_hidden_layers", 1, 4)
    hidden_layers = []
    for i in range(num_layers):
        num_features = trial.suggest_int(f"num_features_layer_{i}", 20, 150)
        hidden_layers.append(num_features)

    # Generate tasks from the signal
    args["k_sup"], args["k_que"] = sup_shots, que_shots
    train_tasks = generate_tasks(train_signals, bkg_file, sup_shots, que_shots)
    val_tasks = generate_tasks(val_signals, bkg_file, sup_shots, que_shots)

    # Choose PyTorch device and create the model
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = Meta(args, get_model(hidden_layers), device).to(device)
    
    # Setup Weights and Biases logger, config hyperparams and watch model
    wandb.init(project="Meta-HEP")
    name = f"K{sup_shots}Q{que_shots}-HL{num_layers}-"
    for i in range(num_layers):
        features = hidden_layers[i]
        name += f"F{features}"
    wandb.run.name = name
    wandb.config.update(args)
    wandb.watch(model)
    print(f"RUN NAME: {name}")

    # Fit the model and return best loss
    return fit(model, train_tasks, val_tasks, args)

# Define optimization parameters

In [ ]:
# Define study arguments
args = {
    "epochs": 1000,
    "epoch_steps": 1000,
    "eval_steps": 200,
    "patience": 12,
    "meta_lr": 1e-3,
    "update_lr": 1e-2,
    "lr_type": "vector",
    "seed": 42
}

# Gather the tasks to run the optimization

In [ ]:
# Datapath and background file
datapath = "processed-data/"
bkg_file = datapath + "bkg.h5"

# Signal files for each split
train_signals = ["hg3000_hq1000", "hg3000_hq1400", "wohg_hq1200"]
val_signals = ["hg3000_hq1200", "wohg_hq1000"]
test_signals = ["wohg_hq1400", "fcnc"]

# Add datapath and extention to files for each split
train_signals = [datapath + p + ".h5" for p in train_signals]
val_signals = [datapath + p + ".h5" for p in val_signals]
test_signals = [datapath + p + ".h5" for p in test_signals]

# Create study and find optimum model architecture

In [ ]:
study_name = "Fixed K-support and K-query optimization"
study = opt.create_study(study_name=study_name, storage='sqlite:///meta-model.db', load_if_exists=True)
objective = lambda trial: optimize(trial, train_signals, val_signals, bkg_file, args)
study.optimize(objective, n_trials=1)
